In [ ]:
import os, warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LinearRegression, LogisticRegressionCV, RidgeClassifierCV, RidgeCV, LassoCV, ElasticNetCV
from sklearn import tree, svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import glob

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics
from sklearn.metrics import roc_curve

SEED = 2021 #random seed #
num_CV = 5  # number of crossvalidation #
n_splits = 5
repeat = 100
test_size = 0.3 # test set ratio #
max_iter = 100000 # max iteration for SVM #
alphas = np.logspace(-4, 4, 17)

In [3]:
X_ADC = pd.read_csv("results/binWidth/ADC_binWidth_32.csv")
X_T1 = pd.read_csv("results/binWidth/T1GD_binWidth_32.csv") 
X_T2 = pd.read_csv("results/binWidth/T2_binWidth_32.csv")

X_full = X_ADC.merge(X_T1, on='ID').merge(X_T2, on='ID').iloc[:, 1:] #full set
y = pd.read_csv("answer/answerADC.csv").iloc[:,1 ]

In [4]:
datasets = ['X_adc_T1_T2', 'X_adc_T1', 'X_adc_T2', 'X_adc', 'X_T1_T2']
oversamples = ['original', 'ros', 'smote', 'adasyn', 'bsmote', 'svmsmote'] #'ksmote', 
methods = ['linear', 'logistic', 'ridge', 'lasso', 'elastic', 'decisiontree', 'randomforest', 'adaboost', 'svm-linear', 'svm-poly', 'svm-rbf', 'fullyconnected']
col_names = []
for o in oversamples:
    for m in methods:
        col_names.append("{}-{}".format(o, m))
for d, dataset in enumerate(datasets):
    print ("[ {} / {} ]".format(d, len(datasets)), "with dataset : ", dataset, )
    if dataset == 'X_adc_T1_T2':
        X = X_full
    elif dataset == 'X_adc_T1':
        X = X_full.iloc[:, :214]
    elif dataset == 'X_adc_T2':
        X = X_full.iloc[:, np.r_[0:107, 214:321]]
    elif dataset == 'X_adc':
        X = X_full.iloc[:, :107]
    elif dataset == 'X_T1_T2':
        X = X_full.iloc[:,107:321]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify = y, random_state=SEED)
    df_train_result = pd.DataFrame([], columns = col_names)
    df_test_result = pd.DataFrame([], columns = col_names)
    pbar = tqdm(total=repeat*n_splits*len(oversamples)*len(methods))
    for seed in range(repeat):
        col_train_values = []
        col_test_values = []
        ros = RandomOverSampler(sampling_strategy='auto', random_state=seed)
        sm = SMOTE(sampling_strategy='auto', random_state=seed)
        adas = ADASYN(sampling_strategy='auto', random_state=seed)
        bsm = BorderlineSMOTE(sampling_strategy='auto', random_state=seed)
        ksm = KMeansSMOTE(sampling_strategy='auto', random_state=seed)
        ssm = SVMSMOTE(sampling_strategy='auto', random_state=seed)
        for _, oversample in enumerate(oversamples):
            if oversample == 'original':
                X_sampled, y_sampled = X_train, y_train
            elif oversample == 'ros':
                X_sampled, y_sampled = ros.fit_resample(X_train,y_train)
            elif oversample == 'smote':
                X_sampled, y_sampled = sm.fit_resample(X_train,y_train)
            elif oversample == 'adasyn':
                X_sampled, y_sampled = adas.fit_resample(X_train,y_train)
            elif oversample == 'bsmote':
                X_sampled, y_sampled = bsm.fit_resample(X_train,y_train)
            elif oversample == 'ksmote':
                X_sampled, y_sampled = ksm.fit_resample(X_train,y_train)
            elif oversample == 'svmsmote':
                X_sampled, y_sampled = ssm.fit_resample(X_train,y_train)
            for _, method in enumerate(methods):
                cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
                AUC_trains = 0
                AUC_tests = 0
                for _, (train, val) in enumerate(cv.split(X_sampled, y_sampled)):
                    if method == 'fullyconnected':
                        sc = StandardScaler()
                        X_stand = sc.fit_transform(X_sampled)
                        X_stand_test = sc.transform(X_test)
                        classifier = Sequential()
                        classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu', input_dim = X.shape[1]))
                        classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
                        classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
                        classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
                        classifier.fit(X_stand[train], y_sampled.to_numpy()[train], batch_size = 10, epochs = 2)
                        preds_train = classifier.predict(X_stand)
                        preds_test = classifier.predict(X_stand_test)
                    else:
                        if method == 'linear':
                            classifier = LinearRegression()
                        elif method == 'logistic':
                            classifier = LogisticRegressionCV(cv = num_CV, random_state=seed)
                        elif method == 'ridge':
                            classifier = RidgeCV(alphas = alphas, cv = num_CV)
                        elif method =='lasso':
                            classifier = LassoCV(alphas = alphas, cv = num_CV, random_state=seed)
                        elif method =='elastic':
                            classifier = ElasticNetCV(alphas=alphas, l1_ratio=0.5, cv = num_CV, random_state=seed)
                        elif method == 'decisiontree':
                            classifier == tree.DecisionTreeClassifier()
                        elif method == 'randomforest':
                            classifier = RandomForestClassifier(max_depth=4, random_state=seed)
                        elif method == 'adaboost':
                            classifier = AdaBoostClassifier(n_estimators=1000, random_state=seed)
                        elif method == 'svm-linear':
                            classifier = svm.SVC(kernel='linear', max_iter = max_iter)
                        elif method == 'svm-poly':
                            classifier = svm.SVC(kernel='poly', max_iter = max_iter)
                        elif method == 'svm-rbf':
                            classifier = svm.SVC(kernel='rbf', max_iter = max_iter)
                        classifier.fit(X_sampled.to_numpy()[train], y_sampled.to_numpy()[train])
                        preds_train = classifier.predict(X_sampled)
                        preds_test = classifier.predict(X_test)
                    performance_train = roc_auc_score(y_sampled, preds_train)
                    performance_test = roc_auc_score(y_test, preds_test)
                    AUC_trains += performance_train
                    AUC_tests += performance_test
                    pbar.update(1)
                col_train_values.append(AUC_trains/n_splits)
                col_test_values.append(AUC_tests/n_splits)
        df_train_result.loc[seed] = col_train_values
        df_test_result.loc[seed] = col_test_values
    df_train_result.to_excel("{}-{}-train.xls".format(dataset, repeat))
    print("saved: ", "{}-{}-train.xls".format(dataset, repeat))
    df_test_result.to_excel("{}-{}-test.xls".format(dataset, repeat))
    print("saved: ", "{}-{}-test.xls".format(dataset, repeat))

[ 0 / 1 ] with dataset :  X_T1_T2


Train on 35 samples
Epoch 1/2
35/35 [==============================] - 1s 28ms/sample - loss: 0.6923 - accuracy: 0.6000
Epoch 2/2
35/35 [==============================] - 0s 543us/sample - loss: 0.6837 - accuracy: 0.8571
Train on 35 samples
Epoch 1/2
35/35 [==============================] - 0s 13ms/sample - loss: 0.6941 - accuracy: 0.3714
Epoch 2/2
35/35 [==============================] - 0s 457us/sample - loss: 0.6862 - accuracy: 0.8571
Train on 35 samples
Epoch 1/2
35/35 [==============================] - 0s 13ms/sample - loss: 0.6896 - accuracy: 0.8571
Epoch 2/2
35/35 [==============================] - 0s 542us/sample - loss: 0.6737 - accuracy: 0.8571
Train on 35 samples
Epoch 1/2
35/35 [==============================] - 0s 11ms/sample - loss: 0.6881 - accuracy: 0.8857
Epoch 2/2
35/35 [==============================] - 0s 428us/sample - loss: 0.6671 - accuracy: 0.8857
Train on 36 samples
Epoch 1/2
36/36 [==============================] - 0s 14ms/sample - loss: 0.6881 - accuracy: 0.8

61/61 [==============================] - 0s 6ms/sample - loss: 0.6909 - accuracy: 0.5410
Epoch 2/2
61/61 [==============================] - 0s 429us/sample - loss: 0.6783 - accuracy: 0.8852
Train on 59 samples
Epoch 1/2
59/59 [==============================] - 0s 7ms/sample - loss: 0.6876 - accuracy: 0.7288
Epoch 2/2
59/59 [==============================] - 0s 419us/sample - loss: 0.6684 - accuracy: 0.8475
Train on 59 samples
Epoch 1/2
59/59 [==============================] - 0s 7ms/sample - loss: 0.6896 - accuracy: 0.7288
Epoch 2/2
59/59 [==============================] - 0s 458us/sample - loss: 0.6746 - accuracy: 0.8644
Train on 59 samples
Epoch 1/2
59/59 [==============================] - 0s 7ms/sample - loss: 0.6879 - accuracy: 0.7288
Epoch 2/2
59/59 [==============================] - 0s 373us/sample - loss: 0.6672 - accuracy: 0.7627
Train on 59 samples
Epoch 1/2
59/59 [==============================] - 0s 7ms/sample - loss: 0.6880 - accuracy: 0.6949
Epoch 2/2
59/59 [==============

47/47 [==============================] - 0s 8ms/sample - loss: 0.6892 - accuracy: 0.5745
Epoch 2/2
47/47 [==============================] - 0s 319us/sample - loss: 0.6715 - accuracy: 0.9149
Train on 48 samples
Epoch 1/2
48/48 [==============================] - 0s 8ms/sample - loss: 0.6916 - accuracy: 0.6667
Epoch 2/2
48/48 [==============================] - 0s 333us/sample - loss: 0.6831 - accuracy: 0.8542
Train on 35 samples
Epoch 1/2
35/35 [==============================] - 0s 11ms/sample - loss: 0.6899 - accuracy: 0.7143
Epoch 2/2
35/35 [==============================] - 0s 372us/sample - loss: 0.6736 - accuracy: 0.8571
Train on 35 samples
Epoch 1/2
35/35 [==============================] - 0s 11ms/sample - loss: 0.6874 - accuracy: 0.8286
Epoch 2/2
35/35 [==============================] - 0s 343us/sample - loss: 0.6677 - accuracy: 0.8571
Train on 35 samples
Epoch 1/2
35/35 [==============================] - 0s 11ms/sample - loss: 0.6921 - accuracy: 0.5714
Epoch 2/2
35/35 [===========

61/61 [==============================] - 0s 6ms/sample - loss: 0.6867 - accuracy: 0.5902
Epoch 2/2
61/61 [==============================] - 0s 363us/sample - loss: 0.6655 - accuracy: 0.6885
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6898 - accuracy: 0.5902
Epoch 2/2
61/61 [==============================] - 0s 438us/sample - loss: 0.6712 - accuracy: 0.8033
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6877 - accuracy: 0.5902
Epoch 2/2
61/61 [==============================] - 0s 393us/sample - loss: 0.6652 - accuracy: 0.8197
Train on 59 samples
Epoch 1/2
59/59 [==============================] - 0s 6ms/sample - loss: 0.6885 - accuracy: 0.7797
Epoch 2/2
59/59 [==============================] - 0s 322us/sample - loss: 0.6705 - accuracy: 0.8983
Train on 59 samples
Epoch 1/2
59/59 [==============================] - 0s 7ms/sample - loss: 0.6888 - accuracy: 0.6102
Epoch 2/2
59/59 [==============

...

61/61 [==============================] - 0s 6ms/sample - loss: 0.6881 - accuracy: 0.8033
Epoch 2/2
61/61 [==============================] - 0s 410us/sample - loss: 0.6630 - accuracy: 0.8361
Train on 44 samples
Epoch 1/2
44/44 [==============================] - 0s 9ms/sample - loss: 0.6887 - accuracy: 0.7500
Epoch 2/2
44/44 [==============================] - 0s 465us/sample - loss: 0.6678 - accuracy: 0.9318
Train on 44 samples
Epoch 1/2
44/44 [==============================] - 0s 9ms/sample - loss: 0.6877 - accuracy: 0.7500
Epoch 2/2
44/44 [==============================] - 0s 409us/sample - loss: 0.6688 - accuracy: 0.8182
Train on 44 samples
Epoch 1/2
44/44 [==============================] - 0s 9ms/sample - loss: 0.6894 - accuracy: 0.7955
Epoch 2/2
44/44 [==============================] - 0s 364us/sample - loss: 0.6741 - accuracy: 0.9318
Train on 44 samples
Epoch 1/2
44/44 [==============================] - 0s 9ms/sample - loss: 0.6906 - accuracy: 0.7500
Epoch 2/2
44/44 [==============

61/61 [==============================] - 0s 6ms/sample - loss: 0.6870 - accuracy: 0.7705
Epoch 2/2
61/61 [==============================] - 0s 442us/sample - loss: 0.6681 - accuracy: 0.8525
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6902 - accuracy: 0.5574
Epoch 2/2
61/61 [==============================] - 0s 426us/sample - loss: 0.6745 - accuracy: 0.7869
Train on 60 samples
Epoch 1/2
60/60 [==============================] - 0s 6ms/sample - loss: 0.6886 - accuracy: 0.7833
Epoch 2/2
60/60 [==============================] - 0s 266us/sample - loss: 0.6656 - accuracy: 0.8667
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 7ms/sample - loss: 0.6852 - accuracy: 0.7705
Epoch 2/2
61/61 [==============================] - 0s 384us/sample - loss: 0.6562 - accuracy: 0.8525
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 7ms/sample - loss: 0.6867 - accuracy: 0.7213
Epoch 2/2
61/61 [==============

61/61 [==============================] - 0s 7ms/sample - loss: 0.6879 - accuracy: 0.7213
Epoch 2/2
61/61 [==============================] - 0s 432us/sample - loss: 0.6674 - accuracy: 0.7869
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6871 - accuracy: 0.6557
Epoch 2/2
61/61 [==============================] - 0s 410us/sample - loss: 0.6687 - accuracy: 0.7705
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6914 - accuracy: 0.5902
Epoch 2/2
61/61 [==============================] - 0s 449us/sample - loss: 0.6774 - accuracy: 0.9016
Train on 48 samples
Epoch 1/2
48/48 [==============================] - 0s 8ms/sample - loss: 0.6878 - accuracy: 0.7083
Epoch 2/2
48/48 [==============================] - 0s 395us/sample - loss: 0.6653 - accuracy: 0.8958
Train on 49 samples
Epoch 1/2
49/49 [==============================] - 0s 8ms/sample - loss: 0.6876 - accuracy: 0.5510
Epoch 2/2
49/49 [==============

61/61 [==============================] - 0s 6ms/sample - loss: 0.6886 - accuracy: 0.7541
Epoch 2/2
61/61 [==============================] - 0s 508us/sample - loss: 0.6701 - accuracy: 0.7869
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6899 - accuracy: 0.6885
Epoch 2/2
61/61 [==============================] - 0s 443us/sample - loss: 0.6699 - accuracy: 0.8033
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6909 - accuracy: 0.6393
Epoch 2/2
61/61 [==============================] - 0s 426us/sample - loss: 0.6785 - accuracy: 0.8197
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6891 - accuracy: 0.6230
Epoch 2/2
61/61 [==============================] - 0s 426us/sample - loss: 0.6709 - accuracy: 0.7541
Train on 60 samples
Epoch 1/2
60/60 [==============================] - 0s 6ms/sample - loss: 0.6883 - accuracy: 0.8333
Epoch 2/2
60/60 [==============

60/60 [==============================] - 0s 6ms/sample - loss: 0.6872 - accuracy: 0.7167
Epoch 2/2
60/60 [==============================] - 0s 333us/sample - loss: 0.6694 - accuracy: 0.8667
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 7ms/sample - loss: 0.6841 - accuracy: 0.7869
Epoch 2/2
61/61 [==============================] - 0s 459us/sample - loss: 0.6449 - accuracy: 0.8689
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 7ms/sample - loss: 0.6871 - accuracy: 0.7213
Epoch 2/2
61/61 [==============================] - 0s 467us/sample - loss: 0.6650 - accuracy: 0.8689
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6874 - accuracy: 0.7541
Epoch 2/2
61/61 [==============================] - 0s 508us/sample - loss: 0.6568 - accuracy: 0.8852
Train on 61 samples
Epoch 1/2
61/61 [==============================] - 0s 6ms/sample - loss: 0.6893 - accuracy: 0.6557
Epoch 2/2
61/61 [==============